In [1]:
import weave
# weave.capture_weave_server_logs()

# weave.use_fixed_server_port()

<OpDef: root-string> root-string None
<OpDef: string-lastLetter> string-lastLetter None
<OpDef: string-set> string-set None
<OpDef: number-set> number-set None
<OpDef: number-add> number-add None
<OpDef: number-sub> number-sub None
<OpDef: number-mult> number-mult None
<OpDef: number-div> number-div None
<OpDef: number-equal> number-equal None
<OpDef: number-notEqual> number-notEqual None
<OpDef: number-less> number-less None
<OpDef: number-greater> number-greater None
<OpDef: number-lessEqual> number-lessEqual None
<OpDef: number-greaterEqual> number-greaterEqual None
<OpDef: pick> pick None
<OpDef: pick> pick None
<OpDef: dict> dict None
<OpDef: merge> merge None
<OpDef: index> index None
<OpDef: count> count None
<OpDef: index> index None
<OpDef: offset> offset None
<OpDef: limit> limit None
<OpDef: map> map None
<OpDef: filter> filter None
<OpDef: groupby> groupby None
<OpDef: group-groupkey> group-groupkey None
<OpDef: list-indexCheckpoint> list-indexCheckpoint None
<OpDef: tag-in

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

# @weave.op(
#     name="root-sin_image",
#     render_info={"type": "function"},
#     input_type={"f": weave.types.Int()},
#     output_type=weave.types.FileType(weave.types.Const(weave.types.String(), "png")),
# )
# def sin_image(f):    
#     x = np.arange(0, f * np.pi, 0.1)
#     y = np.sin(x)
#     path = "/tmp/my-%s.png" % random.randrange(0, 1000000)
#     plt.rcParams["figure.figsize"] = (22, 7)
#     plt.plot(x, y)
#     plt.savefig(path)
#     plt.close()
#     return weave.ops.LocalFile(path)


@weave.op(
    name="root-sin_val",
    render_info={"type": "function"},
)
def sin_val(x: float) -> float:    
     return float(np.sin(x))

<OpDef: root-sin_val> root-sin_val None


In [3]:
x = weave.save(7, name='my-number')

# im = sin_image(x + 2)
weave.use(sin_val(x))

7 my-number None


Exception: ('invalid scheme ', 'my-number/93936b8ceaec82097f911d4d0d04e174')

In [ ]:
import json
class RunType(weave.types.BasicType):
    name = 'run'

@weave.op(
    name="root-run_accuracy_barchart",
    render_info={"type": "function"},
    input_type={"runs": weave.types.List(weave.ops.RunType())},
    output_type=weave.types.FileType(weave.types.Const(weave.types.String(), 'png'))
)
def run_accuracy_barchart(runs):
    names = [r['name'] for r in runs]
    xs = range(len(runs))
    ys = [r['summary'].get('acc') for r in runs]
    
    plt.rcParams["figure.figsize"] = (22, 5)
    plt.bar(xs, ys, align='center', alpha=0.5)
    plt.xticks(xs, names)
    plt.ylabel('acc')
    plt.title('Run accuracies')
    path = "/tmp/my-%s.png" % random.randrange(0, 1000000)
    plt.savefig(path)
    plt.close()
    return weave.ops.LocalFile(path) 

In [ ]:
runs = weave.ops.project('shawn', 'fasion-sweep').runs()
run_accuracy_barchart(runs)